# Exploring Dynamic Machine Learning Methods for FX Portfolio Risk Management

In [8]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import *
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
sys.path.append('..')
from Trading import *

# Decision Trees

In [10]:
filepath = "/Users/andresvillacis/Documents/GitHub/capstone_proj/Data/FX_Test_USD-per-FX_Chicago.csv"
hola = Batch(start='1999-01-01', end='2000-01-01',filepath=filepath)
hola.px_data


TypeError: __init__() got an unexpected keyword argument 'filepath'

# Random Forests